In [ ]:
import requests

from random import randint

# disperse
from ecc import PrivateKey
from helper import decode_base58, hash256, little_endian_to_int
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

passphrase = b'jimmy@programmingblockchain.com secret passphrase'
secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)
my_address = priv.point.address(testnet=True)
print(my_address)
target_amount = 0.1
fee = 0.001

response = requests.get('https://testnet.blockchain.info/unspent?active={}'.format(my_address))
utxos = response.json()['unspent_outputs']
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')

student_addresses = '''mseRGXB89UTFVkWJhTRTzzZ9Ujj4ZPbGK5'''
# initialize inputs
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['tx_hash_big_endian'])
    prev_index = utxo['tx_output_n']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))
# initialize outputs
tx_outs = []
output_amount = 0
for target_address in student_addresses.split():
    # decode the hash160 from the target address
    h160 = decode_base58(target_address)
    # convert hash160 to p2pkh script
    script_pubkey = p2pkh_script(h160)
    # convert target amount to satoshis (multiply by 100 million)
    target_satoshis = int(target_amount * 100000000)
    output_amount += target_satoshis
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_satoshis,
        script_pubkey=script_pubkey,
    ))

change_satoshis = total - int(fee*100000000) - output_amount
h160 = decode_base58(my_address)
script_pubkey = p2pkh_script(h160)
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=script_pubkey))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, priv)

# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())

In [ ]:
import requests

from random import randint

# disperse
from ecc import PrivateKey
from helper import decode_base58, hash256, little_endian_to_int
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

passphrase = b'jimmy@programmingblockchain.com secret passphrase'

secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)
my_address = priv.point.address(testnet=True)
target_amount = 0.1
fee = 0.001

response = requests.get('https://testnet.blockchain.info/unspent?active={}'.format(my_address))
utxos = response.json()['unspent_outputs']
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')

student_addresses = 'mseRGXB89UTFVkWJhTRTzzZ9Ujj4ZPbGK5'
# initialize inputs
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['tx_hash_big_endian'])
    prev_index = utxo['tx_output_n']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))

# initialize outputs
tx_outs = []
output_amount = 0
for target_address in student_addresses.split():
    # decode the hash160 from the target address
    h160 = decode_base58(target_address)
    # convert hash160 to p2pkh script
    script_pubkey = p2pkh_script(h160)
    # convert target amount to satoshis (multiply by 100 million)
    target_satoshis = randint(5000000, 20000000)
    output_amount += target_satoshis
    # create a new tx output for target
    tx_outs.append(TxOut(target_satoshis, script_pubkey))

change_satoshis = total - int((fee)*100000000) - output_amount

h160 = decode_base58(my_address)
script_pubkey = p2pkh_script(h160)
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=script_pubkey))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, priv)

# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())

In [ ]:
import requests

from random import randint

# disperse
from ecc import PrivateKey
from helper import decode_base58, hash256, little_endian_to_int
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

passphrase = b'Jimmy Song'
secret = little_endian_to_int(hash256(passphrase))
priv = PrivateKey(secret=secret)
my_address = priv.point.address(testnet=True)
print(my_address)
fee = 0.001

response = requests.get('https://testnet.blockchain.info/unspent?active={}'.format(my_address))
utxos = response.json()['unspent_outputs']
if (len(utxos) == 0):
    raise RuntimeError('no outputs!')

consolidation_address = '''mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv'''
# initialize inputs
tx_ins = []
total = 0
for utxo in utxos:
    if utxo['confirmations'] == 0:
        continue
    if utxo['tx_hash_big_endian'] in (
        '3d553a0343c160f7429f19bab57f0efdf146847f279dcb86a881319e2e21ff4a',
        '869f7d3ec5a910f486a203f52980cd60df4d3d586d738fad47858757406bd04f',
    ):
        continue
    
    prev_tx = bytes.fromhex(utxo['tx_hash_big_endian'])
    prev_index = utxo['tx_output_n']
    total += utxo['value']
    # create a new tx input
    tx_ins.append(TxIn(prev_tx, prev_index))
# initialize outputs
tx_outs = []
# decode the hash160 from the target address
h160 = decode_base58(consolidation_address)
# convert hash160 to p2pkh script
script_pubkey = p2pkh_script(h160)
target_satoshis = total - int(fee*100000000)
print(target_satoshis)
# create a new tx output for target
tx_outs.append(TxOut(
    amount=target_satoshis,
    script_pubkey=script_pubkey,
))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, priv)

# SANITY CHECK: fee is reasonable
if tx_obj.fee() > 0.05*100000000 or tx_obj.fee() <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())